In [1]:
import os
import re
import json
import UnityPy
from collections import defaultdict
from datetime import datetime

In [2]:
# game data folder location
urbek_data = r"H:\Steam\steamapps\common\Urbek\Urbek_Data\resources.assets"
modified_date = datetime.fromtimestamp(os.path.getmtime(fr"{urbek_data}")).strftime('%Y-%m-%d')
modified_date

'2023-02-04'

In [3]:
# utils
gray = lambda x: f'<span class="gray">{x}</span>'
gray_self = lambda x, codeName: str(x).replace(f"'{codeName}'", gray(f"'{codeName}'"))
plus_minus = lambda x: f'<span class="plus">+{x}</span>' if x > 0 else f'<span class="minus">{x}</span>' if x < 0 else '0'
more_less = lambda biggerThan, x: str(x) if biggerThan else f'<span class="less">{"" if str(x)[0] == "0" else "≤"}{x}</span>'
extracted = lambda x: f'<span class="extracted">{x}</span>'
tr = lambda arr: ''.join(f'<tr class="{cls}">{i}</tr>' for i, cls in arr)
th = lambda arr: ''.join(f'<th>{i}</th>' for i in arr)
td = lambda arr: ''.join(f'<td>{i}</td>' for i in arr)
apos = lambda x: str(x).replace("'", "‛")
translate = lambda x, lang='en': f'<span class="translate" title=\'{apos(json.dumps(x, indent=2, ensure_ascii=False))}\'>{x.get(lang, "?")}</span>'
option = lambda arr: ''.join(f"<option>{i}</option>" for i in arr)

In [4]:
# load data file from game installation
env = UnityPy.load(urbek_data)

rules = {}
translations = {}
for obj in env.objects:
    if obj.type.name != "TextAsset": continue
    data = obj.read()
    # download json files for browsing:
    os.makedirs("./data", exist_ok=True)
    if data.name == "gameValues":
        for item in json.loads(data.text):
            del item["INFO"]
            translations[item["id"]] = item
    elif re.match(r"reglasBioma_\d$", data.name) and \
       re.search(r"localVariables", data.text):
        rules[data.name] = json.loads(data.text)
    else:
        continue
    with open(f"./data/{data.name}.json", "wb") as f:
        f.write(data.script)
del env, obj, data

# explore list of languages
languages = [i for i in translations["resources_comida"].keys() if i not in ['id', 'INFO']]
languages

['en', 'es', 'fr', 'zh-CN', 'pl', 'de', 'pt', 'ru', 'ja', 'it', 'ko']

In [5]:
labels = dict((k, v)
              for k, v in translations.items()
              if not re.search(r"text|desc|name|effect|t_|nr_", k))

# explore list of (non-empty) biomes
biomes = dict(sorted([(k, re.sub(r"<b>|</b>[\s\S]*", "", v["description"])) for k, v in rules.items()]))
biomes

{'reglasBioma_0': 'Temperate',
 'reglasBioma_1': 'Desert',
 'reglasBioma_2': 'Archipelago',
 'reglasBioma_3': 'Forest',
 'reglasBioma_4': 'Ruins',
 'reglasBioma_5': 'Snow',
 'reglasBioma_6': 'Temperate'}

In [27]:
biomes["reglasBioma_5"] = "Mountains DLC"
biomes["reglasBioma_6"] = "War DLC"

columns_resources = {}
columns_lv = {}
for biome, biome_label in biomes.items():
    data = rules[biome]
    columns_resources |= {item.split(',')[0]: 1 for item in data["resources"]}
    columns_lv |= {item.split(',')[0]: 1 for item in data["localVariables"]}
columns_resources = columns_resources.keys()
columns_lv = columns_lv.keys()

buildings = []
order_map = {}
index = 0
prev_population = 0
for biome, biome_label in biomes.items():
    data = rules[biome]
    updatesFromDict = defaultdict(list)
    for building in data["construs"]:
        if up := building.get('updates'):
            for u in up:
                updatesFromDict[u].append(building['codeName'])

    for building in data["construs"]:
        category = building.get('category')
        if category == 'camino' and not building.get('cost'): continue
        
        category = translate(labels.get(f'bg_{category}', {"id": category})) if category else ""
        codeName = building['codeName']
        
        label_all = labels.get(f'b_{codeName}', {})
        label = label_all.get('en', '???')
        distanceToRoad = building.get('distanceToRoad', '')
        if distanceToRoad == 3:
            distanceToRoad = gray(distanceToRoad)
        
        updates = gray_self(building.get('updates', ''), codeName)
        updatesFrom = gray_self(updatesFromDict.get(codeName, ''), codeName)
        buildingsNeeded = [more_less(x['biggerThan'], f"{x['q']} {x['building']} ⊙{x['radio'] if x['radio'] else '∞'}")
                           for x in building.get('buildingsNeeded')
                          ] if building.get('buildingsNeeded') else ''
        unblock = [f"{x['q']} {x.get('gloVar', x.get('nBuildings', '?'))}"
                   for x in building.get('unblock')
                  ] if building.get('unblock') else ''
        unblock_population = [x['q']
                              for x in building.get('unblock') if x.get('gloVar') == 'poblacion'
                             ] if building.get('unblock') else [0]
        cost = [f"{x['q']} {x.get('rec')}"
                for x in building.get('cost')
               ] if building.get('cost') else ''
        minModels = min([re.sub(r"^[^,]*,|,E?$", "", x) for x in building.get('models') if x
                        ]) if building.get('models') else ''
        if minModels == '1,1':
            minModels = gray(minModels)
        byBuilding = '🏠' if building.get('byBuilding') else ''

        lvn = [next((more_less(
            x['biggerThan'],
            f"{gray(x['q']) if x['q'] == 1 else x['q']}"
            f"{'□ ' + str(x['threshold']).rjust(2) if 'threshold' in x else ''}"
            f" ⊙{x['radio']}"
            ) for x in building.get('localVariablesNeeded', []) if x['localVariable'] == col), "")
              for col in columns_lv]
        lv = [next((plus_minus(x['q'])
                     for x in building.get('localVariables', [])
                     if x['locVar'] == col), "")
              for col in columns_lv]
        rn = [next((more_less(x['biggerThan'], f"{x['q']} ⊙{x['radio']}")
                    for x in building.get('resourcesNeeded', [])
                    if x['resource'] == col), "")
              for col in columns_resources]
        rx = [next((extracted(f"-{x['q']} ⊙{x['distance']}")
                    for x in ([building.get('resourceExtraction')] if building.get('resourceExtraction') else [])
                    if x['resource'] == col), "")
              for col in columns_resources]
        r = [next((plus_minus(x['q'])
                   for x in building.get('produces', [])
                   if x['rec'] == col), "")
             for col in columns_resources]

        lvnlv = [f"{a}{' '+b.rjust(3) if b else ''}" for a, b in zip(lvn, lv)]
        rnrxr = [f"{a}{' '+b.rjust(3) if b else ''}{' '+c.rjust(3) if c else ''}" for a, b, c in zip(rn, rx, r)]
        
        if any([updates, unblock, *rn, *rx, *r, *lvn, *lv]):
            if len(unblock_population):
                unblock_population = unblock_population[0]
                prev_population = unblock_population   
            else:
                unblock_population = prev_population
            if codeName not in order_map:
                order_map[codeName] = 1000 * unblock_population + index
                index += 1
            buildings.append([biome_label,
                              translate(label_all),
                              distanceToRoad,
                              minModels,
                              byBuilding,
                              *lvnlv,
                              *rnrxr,
                              codeName,
                              buildingsNeeded,
                              unblock,
                              cost,
                              updatesFrom,
                              updates,
                              category,
                              order_map[codeName],
                             ])

buildings.sort(key=lambda i: i[-1])
# visualize buildings
columns = ["Biome", "Building name", "distanceToRoad", "smallest size", "byBuilding",
           *[f"{translate(labels[f'lv_{c}'])}  ({gray(c)})" for c in columns_lv],
           *[f"{translate(labels[f'resources_{c}'])}  ({gray(c)})" for c in columns_resources],
           "codeName", "buildingsNeeded", "unblock", "cost", "updatesFrom", "updates", "category",
          ]

search = [
    f'<select>{option(["", *biomes.values()])}</select>',
    '<input>', '', '',
    *['<input type="checkbox">'] * (len(columns_lv) + len(columns_resources) + 1),
    *['<input>'] * 7,
]

html = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Urberk buildings</title>
  <meta name="viewport"
        content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0">
  <style>
  * {
      box-sizing: border-box;
  }
  body {
      margin: 0 0 60px;
      color: white;
      font: .9rem sans-serif;
      background: #252525;
      width: fit-content;
  }
  .intro {
      position: sticky;
      left: 0;
      width: 100vw;
  }
  table {
      position: relative;
      border-collapse: collapse;
  }
  @media (min-height: 600px) {
      thead {
          position: sticky;
          top: 0;
          z-index: 1;
      }
  }
  thead tr {
      background: black;
  }
  thead tr:first-child {
      height: 225px;
  }
  thead th {
      white-space: pre;
      vertical-align: bottom;
      text-align: left;
      padding: 5px 5px 0;
  }
  thead td {
      padding: 0 5px 5px;
  }
  thead input,
  thead select {
      width: 100%;
      padding: 0;
  }
  thead tr th:nth-child(n+3):not(:nth-last-child(-n+7)) {
      transform-origin: bottom left;
      transform: translateX(20px) rotate(-60deg);
      max-width: 20px;
  }
  tbody tr {display: none;}
  tr {background: #333}
  tr.even {background: #222}
  tr:hover {background: black}
  tr.T td:nth-child(2) {color: yellowgreen}
  tr.D td:nth-child(2) {color: goldenrod}
  tr.A td:nth-child(2) {color: dodgerblue}
  tr.F td:nth-child(2) {color: forestgreen}
  tr.R td:nth-child(2) {color: rosybrown}
  tr.M td:nth-child(2) {color: white; text-shadow: 0 0 2px #fffa}
  tr.W td:nth-child(2) {color: red}
  td {
      white-space: pre;
      border-left: 1px dotted #444;
      padding: 5px;
  }
  @media (min-width: 600px) {
      th:nth-child(1),
      td:nth-child(1),
      th:nth-child(2),
      td:nth-child(2) {
          position: sticky;
          left: 14px;
          background: inherit;
      }
      th:nth-child(1) {
          left: -60px;
          z-index: 1;
          min-width: 105px;
      }
      th:nth-child(2) {
          z-index: 1;
      }
      td:nth-child(1) {
          left: 0;
          font-family: monospace;
      }
  }
  tbody td:nth-child(n+3) {
      text-align: right;
      font-family: monospace;
  }
  tbody td:nth-last-child(-n+7) {
      text-align: left;
      max-width: 250px;
      overflow: hidden;
      white-space: nowrap;
      text-overflow: ellipsis;
      line-height: 1.2;
  }
  tbody td:nth-last-child(-n+6):not(:last-child):hover {
      position: absolute;
      white-space: pre-wrap;
      background: black;
  }
  h1 { margin: 15px 14px; }
  p, dt { margin-left: 15px; }
  dd { font-family: monospace; }
  a { color: goldenrod; transition: text-decoration-color 200ms; }
  a:hover { text-decoration-color: orange; }
  .gray { color: #999; font-weight: normal; }
  .less { color: goldenrod; }
  .plus { color: #7f7; }
  .minus { color: #f66; }
  .extracted { color: #d6f; }
  </style>
</head>""" + f"""
<body>
  <div class="intro">
    <h1>
      List of buildings from
      <a href="https://store.steampowered.com/app/1411740/Urbek_City_Builder/">
        Urbek&nbsp;City&nbsp;Builder
      </a>
    </h1>
    <p>This is a fan-made page. Game data extracted with UnityPy, from <i>reglasBioma_*</i>
      TextAsset files inside <i>resources.assets</i> ({modified_date}).
    <br>© Copyright of game by Estudios Kremlinois (developer) and RockGame S.A. (publisher).
    <br>© Copyright of this tool (but not game data) by Peter Hozák under MIT License,
      <a href="https://github.com/Aprillion/urbek">https://github.com/Aprillion/urbek</a>.

    <p><b>How to read the table:</b>
    <dl>
      <dt>Village house needs 15 squares with 6 residents or more (on each square) within radius 3
          (7x7 area with house in the centre), and produces 8 residents:
          <dd>15□  6 ⊙3 <span class="plus">+8</span>
      <dt>Villa needs 25 residents or less within radius 4, and produces 6 residents per square
          (24 in total):
          <dd><span class="less">≤25 ⊙4 <span class="plus">+6</span></span>:
      <dt>Minimarket consumes 200 Food:
          <dd><span class="minus">-200</span>
      <dt>Lumbercamp extracts 10 natural resources from trees within radius 5, and produces 15 wood:
          <dd><span class="extracted">-10 ⊙5</span> <span class="plus">+15</span>
      <dt>Rebel house needs that no other Rebel houses and no Police stations are within radius 5:
          <dd>['<span class="less">0 rebelde ⊙5</span>', '<span class="less">0 pacos ⊙5</span>']
    </dl>
    <br>
    <p>Use <b>SHIFT+scroll</b> for horizontal scrolling on desktop.
      On mobile, you can tap a row to highlight it before scrolling (and/or rotate screen).
    <p>
      <label>Select game language: <select id="lang" data-prev="en" autocomplete="off">{option(languages)}</select></label>
      <a href="#?"><i>Reset all filters</i></a>
  </div>
  <table>
    <thead>
      <tr>{th(columns)}</tr>
      <tr>{td(search)}</tr>
    </thead>
    {tr((td(b[:-1]), b[0][0]) for b in buildings)}
  </table>""" + """
  <script>
    const inputEls = [...document.querySelectorAll('thead input, thead select')]
    const filterFieldEntries = [...document.querySelector('thead tr:nth-child(2)').children]
      .map((td, i) => ([`c${i}`, td.children[0]]))
      .filter(([_, el]) => el)
    const filterFields = Object.fromEntries(filterFieldEntries)
    let rows
    
    if (!location.hash) location.hash = '?c0=Temperate,'
    else if (!location.hash.match(',$')) location.hash += ','
    updateFromHash(location.hash.slice(2))
    window.addEventListener("hashchange", () => updateFromHash(location.hash.slice(2)))
    document.addEventListener("change", handleChange)
    
    function handleChange(ev) {
      const el = ev.target
      if (el.id) {
        const v = ev.target.value
        location.hash = v === 'en' ? '?' : `?${el.id}=${v},`
        return
      }
      const column = Array.prototype.indexOf.call(el.parentNode.parentNode.children, el.parentNode)
      let hash = location.hash.slice(2)
      if (el.checked) {
        hash += `c${column},`
      } else {
        hash = hash.replace(`c${column}`, 'x').replace(/x(?:=.+?)?,/, '')
        if (el.value) {
          hash += `c${column}=${el.value},`
        }
      }
      location.hash = `?${hash}`
    }
    function updateFromHash(hash) {
      hash = decodeURIComponent(hash)
      document.title = `Urbek buildings ${hash}`
      const filterEntries = hash.split(',').slice(0, -1).map((kv) => kv.split('='))
      const filter = Object.fromEntries(filterEntries)
      // console.warn(filter)
      if (filter.lang) {
        updateLanguage(filter.lang)
      } else {
        updateLanguage('en')
      }
      for (const el of inputEls) {
        el.checked = false
        el.value = ''
      }
      const filterEntriesInTable = filterEntries.filter(([k]) => k in filterFields)
      for (const [k, v] of filterEntriesInTable) {
        if (filterFields[k].type === 'checkbox') filterFields[k].checked = true
        else if (v) filterFields[k].value = v
      }
      
      updateTable(filterEntriesInTable.map(([k, v]) => [k, new RegExp(v || '.', 'i')]))
    }
    
    function updateLanguage(lang) {
      let el = document.getElementById('lang')
      if (el.dataset.prev === lang) return
      
      el.value = lang
      el.dataset.prev = lang
      for (el of document.querySelectorAll(".translate")) {
        try {
          const text = JSON.parse(el.getAttribute("title"))[lang]
          el.innerText = text
        } catch (e) {
          console.error(el, e)
        }
      }
      updateRows()
    }
    
    function updateTable(filterEntriesInTable) {
      if (!rows) updateRows()
      for (const row of rows) {
        row.match = filterEntriesInTable.every(([k, v]) => row.texts[k].match(v))
        row.tr.style.display = row.match ? 'table-row' : 'none'
      }
      setTimeout(() => {
        let even = false
        let prev = null
        for (const row of rows) {
          if (!row.match) continue
          if (prev !== row.texts.c1) {
            even = !even
            prev = row.texts.c1
          }
          if (even) row.tr.classList.add('even')
          else row.tr.classList.remove('even')
        }
      })
    }
    
    function updateRows() {
      rows = [...document.querySelectorAll('tbody tr')]
        .map(tr => ({
          tr,
          texts: Object.fromEntries(
            [...tr.children].map((td, i) => ([`c${i}`, td.innerText]))
          )
        }))
    }
  </script>
</body>
</html>
"""
fname = 'docs/index.html'
with open(fname, 'wb') as f:
    f.write(html.encode("utf-8"))

In [9]:
# use `npx live-server docs` for live preview of the output